In [97]:
# this notebook is inspired by @robikscube
#30-06-2022
#dataset amazon-fine-food-reviews

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from tqdm.notebook import tqdm

In [98]:
dataset=pd.read_csv("../input/amazon-fine-food-reviews/Reviews.csv")
dataset=dataset.head(100)
print(dataset.shape)

In [99]:
dataset.head(10)

In [100]:
print(dataset["Text"][1])

In [101]:
exp_sentence=dataset["Text"][1]
print(exp_sentence)

exp_tokens=nltk.word_tokenize(exp_sentence)
print(exp_tokens)

In [102]:
nltk.pos_tag(exp_tokens[:10])

# Using Roberta Model for Sentiment Analysis

In [103]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [104]:
MODEL=f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer=AutoTokenizer.from_pretrained(MODEL)
model=AutoModelForSequenceClassification.from_pretrained(MODEL)

In [105]:
print(exp_sentence)

In [106]:
def spa_roberta(exp_sentence):
    # sentece polarity analysis Roberta
    encoded_text=tokenizer(exp_sentence,return_tensors="pt")
    output=model(**encoded_text)
    scores=output[0][0].detach().numpy()
    scores=softmax(scores)
    scores_dict={
    "roberta_neg":scores[0],
    "roberta_neu":scores[1],
    "roberta_pos":scores[2],
    }
    return scores_dict

In [107]:
print(spa_roberta(exp_sentence))

In [108]:
final_score={}
for i, row in tqdm(dataset.iterrows(),total=len(dataset)):
    try:
        text=row["Text"]
        t_id=row["Id"]
        roberta_score=spa_roberta(text)
        final_score[t_id]=[roberta_score,text]
    except RuntimeError:
        print(f"Broke for id {t_id}")


In [109]:
print(final_score[3])
print(final_score[4])
# This dictionary can be formatted to get proper labels

In [110]:
# print(dataset["Id"][0])

import colorama
from colorama import Fore

In [111]:
for id,value in final_score.items():
    score,text=value
    # print(score,text)
    # print(score["roberta_neg"],score["roberta_pos"])
    if score["roberta_neg"]>score["roberta_pos"]:
        print(Fore.RED + text[:100])
    else:
        print(Fore.GREEN + text[:100])
        
